In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/ALQAC")
os.listdir()

['preprocessing.ipynb',
 'uv.lock',
 '.python-version',
 'pyproject.toml',
 'hello.py',
 '.gitignore',
 'README.md',
 'evaluate.py',
 'indexing.ipynb',
 'utils.py',
 'law_search',
 'ALQAC_2025_data',
 '__pycache__',
 'qdrant',
 'output',
 'evaluation',
 '.git',
 'reranker_models',
 'reranker.ipynb']

In [3]:
!pip install \
  "fastembed" \
  "llama-index" \
  "llama-index-embeddings-huggingface" \
  "llama-index-postprocessor-flag-embedding-reranker" \
  "llama-index-vector-stores-qdrant" \
  "sentence-transformers"

INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 130.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from law_search.vector_db import QdrantCollection
from llama_index.core import Settings

# Hardcoded configurations
MODEL_NAME = "truro7/vn-law-embedding"
# MODEL_CACHE_DIR = "./models"
COLLECTION_NAME = "law_sections"


def setup_embedding_model() -> None:
    """Initialize and setup the embedding model."""

    embed_model = HuggingFaceEmbedding(
        model_name=MODEL_NAME,
        trust_remote_code=True,
        # cache_folder=MODEL_CACHE_DIR,
    )
    Settings.embed_model = embed_model
    Settings.chunk_size = 512
    Settings.db = QdrantCollection(collection_name="law_sections")


In [5]:
# Setup embedidng
setup_embedding_model()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

portuguese.txt: 0.00B [00:00, ?B/s]

russian.txt: 0.00B [00:00, ?B/s]

swedish.txt:   0%|          | 0.00/559 [00:00<?, ?B/s]

turkish.txt:   0%|          | 0.00/260 [00:00<?, ?B/s]

Vector store initialized successfully


In [35]:
TOP_K = 3
SPARSE_TOP_K = 20

retriever_engine = Settings.db._index.as_retriever(
    similarity_top_k=TOP_K,
    sparse_top_k=SPARSE_TOP_K,
    vector_store_query_mode="hybrid",
    # node_postprocessor=[],
)

In [36]:
from llama_index.core.schema import QueryBundle


def retrieve(query, retriever_engine, top_k=2, is_postprocessed=True, threshold=0.5):
    result_nodes = retriever_engine._retrieve(QueryBundle(query_str=query))
    results = []
    for node in result_nodes:
        if not is_postprocessed or node.score >= threshold or top_k == 1:
            results.append(
                {
                    "document": node.node.metadata["doc_id"],
                    "id": node.node.metadata["section_id"],
                    "score": node.score,
                    "text": node.node.text,
                }
            )
    return {"result": results}


query = """
Một người có thể được người có quyền theo quy định của pháp luật yêu cầu Tòa án ra quyết định tuyên bố là đã chết khi người đó biệt tích trong chiến tranh sau 05 năm, kể từ ngày chiến tranh kết thúc mà vẫn không có tin tức xác thực là còn sống, đúng hay sai?
"""
results_json = retrieve(query, retriever_engine, TOP_K, False)
results_json

{'result': [{'document': 'Bộ luật dân sự',
   'id': '71',
   'score': 1.0,
   'text': 'Tuyên bố chết\n\n1. Người có quyền, lợi ích liên quan có thể yêu cầu Tòa án ra quyết định tuyên bố một người là đã chết trong trường hợp sau đây:\n\na) Sau 03 năm, kể từ ngày quyết định tuyên bố mất tích của Tòa án có hiệu lực pháp luật mà vẫn không có tin tức xác thực là còn sống;\n\nb) Biệt tích trong chiến tranh sau 05 năm, kể từ ngày chiến tranh kết thúc mà vẫn không có tin tức xác thực là còn sống;\n\nc) Bị tai nạn hoặc thảm họa, thiên tai mà sau 02 năm, kể từ ngày tai nạn hoặc thảm hoạ, thiên tai đó chấm dứt vẫn không có tin tức xác thực là còn sống, trừ trường hợp pháp luật có quy định khác;\n\nd) Biệt tích 05 năm liền trở lên và không có tin tức xác thực là còn sống; thời hạn này được tính theo quy định tại khoản 1 Điều 68 của Bộ luật này.\n\n2. Căn cứ vào các trường hợp quy định tại khoản 1 Điều này, Tòa án xác định ngày chết của người bị tuyên bố là đã chết.\n\n3. Quyết định của Tòa án tuyê

In [8]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForCausalLM

model_name = "Qwen/Qwen3-Reranker-0.6B"
tokenizer = AutoTokenizer.from_pretrained(
    model_name, cache_dir=f"reranker_models/{model_name}"
)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()
model.to('cuda')


config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151669, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [37]:
import torch


def format_instruction(instruction, query, doc):
    output = "<Instruct>: {instruction}\n<Query>: {query}\n<Document>: {doc}".format(
        instruction=instruction, query=query, doc=doc
    )
    return output


def process_inputs(pairs):
    inputs = tokenizer(
        pairs,
        padding=False,
        truncation="longest_first",
        return_attention_mask=False,
        max_length=max_length - len(prefix_tokens) - len(suffix_tokens),
    )
    for i, ele in enumerate(inputs["input_ids"]):
        inputs["input_ids"][i] = prefix_tokens + ele + suffix_tokens
    inputs = tokenizer.pad(
        inputs, padding=True, return_tensors="pt", max_length=max_length
    )
    for key in inputs:
        inputs[key] = inputs[key].to(model.device)
    return inputs


@torch.no_grad()
def compute_logits(inputs, **kwargs):
    batch_scores = model(**inputs).logits[:, -1, :]
    true_vector = batch_scores[:, token_true_id]
    false_vector = batch_scores[:, token_false_id]
    batch_scores = torch.stack([false_vector, true_vector], dim=1)
    batch_scores = torch.nn.functional.log_softmax(batch_scores, dim=1)
    scores = batch_scores[:, 1].exp().tolist()
    return scores


token_false_id = tokenizer.convert_tokens_to_ids("no")
token_true_id = tokenizer.convert_tokens_to_ids("yes")
tokenizer.padding_side  = 'left'
max_length = 8192

prefix = '<|im_start|>system\nJudge whether the Document meets the requirements based on the Query and the Instruct provided. Note that the answer can only be "yes" or "no".<|im_end|>\n<|im_start|>user\n'
suffix = "<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n"
prefix_tokens = tokenizer.encode(prefix, add_special_tokens=False)
suffix_tokens = tokenizer.encode(suffix, add_special_tokens=False)

In [38]:
task = """Given a  search query, retrieve relevant passages that answer the query. You should output the score for each pair (from 0 to 1).
only give the id and corresponding score. do not explain. Give in format {id: score}. eg {1: 0.8}"""

documents = [f'id={result["id"]} \n {result["text"]}' for result in results_json["result"]]
queries = [query for i in range(len(documents))]
pairs = [format_instruction(task, query, doc) for query, doc in zip(queries, documents)]


In [39]:
from pprint import pprint
pprint(pairs)


['<Instruct>: Given a  search query, retrieve relevant passages that answer '
 'the query. You should output the score for each pair (from 0 to 1). \n'
 'only give the id and corresponding score. do not explain. Give in format '
 '{id: score}. eg {1: 0.8}\n'
 '<Query>: \n'
 'Một người có thể được người có quyền theo quy định của pháp luật yêu cầu Tòa '
 'án ra quyết định tuyên bố là đã chết khi người đó biệt tích trong chiến '
 'tranh sau 05 năm, kể từ ngày chiến tranh kết thúc mà vẫn không có tin tức '
 'xác thực là còn sống, đúng hay sai?\n'
 '\n'
 '<Document>: id=71 \n'
 ' Tuyên bố chết\n'
 '\n'
 '1. Người có quyền, lợi ích liên quan có thể yêu cầu Tòa án ra quyết định '
 'tuyên bố một người là đã chết trong trường hợp sau đây:\n'
 '\n'
 'a) Sau 03 năm, kể từ ngày quyết định tuyên bố mất tích của Tòa án có hiệu '
 'lực pháp luật mà vẫn không có tin tức xác thực là còn sống;\n'
 '\n'
 'b) Biệt tích trong chiến tranh sau 05 năm, kể từ ngày chiến tranh kết thúc '
 'mà vẫn không có tin 

In [42]:
# prompt: normalize scores using softmax

# Get scores from the results_json
scores = [result['score'] for result in results_json['result']]

# Convert scores to a PyTorch tensor
scores_tensor = torch.tensor(scores, dtype=torch.float32)

# Apply softmax
softmax_scores = torch.softmax(scores_tensor, dim=0)

softmax_scores

tensor([0.5193, 0.2472, 0.2335])

In [44]:
# Tokenize the input texts

inputs = process_inputs(pairs)
scores = compute_logits(inputs)
# Convert scores to a PyTorch tensor
scores_tensor = torch.tensor(scores, dtype=torch.float32)

# Apply softmax
softmax_scores = torch.softmax(scores_tensor, dim=0)
print("scores: ", scores)
print("softmax:", softmax_scores)

scores:  [0.9978564381599426, 0.9935508966445923, 0.9646556377410889]
softmax: tensor([0.3375, 0.3360, 0.3265])


In [16]:
def rerank_documents(task, documents, query):
    """
    Reranks documents based on their relevance to a given query using a sequence classification model.

    Args:
        task (str): The task instruction.
        documents (list): A list of document texts to rerank.
        query (str): The search query.
        tokenizer: The tokenizer for the reranking model.
        model: The reranking model.
        prefix_tokens (list): List of tokens to prepend to the input.
        suffix_tokens (list): List of tokens to append to the input.
        token_true_id (int): The token id representing 'yes'.
        token_false_id (int): The token id representing 'no'.
        max_length (int): The maximum length for tokenized inputs.

    Returns:
        list: A list of scores representing the relevance of each document.
    """
    queries = [query for _ in range(len(documents))]
    pairs = [format_instruction(task, query, doc) for query, doc in zip(queries, documents)]

    inputs = process_inputs(pairs, tokenizer, prefix_tokens, suffix_tokens, max_length, model)
    scores = compute_logits(inputs, model, token_true_id, token_false_id)
    return scores


In [17]:
# prompt: sort but keep the index of the scores

sorted_scores_with_indices = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)

# Print the sorted scores and their original indices
print("Sorted scores with indices (original index, score): ", sorted_scores_with_indices)

Sorted scores with indices (original index, score):  [(0, 0.9999151825904846), (1, 0.9990034699440002), (2, 0.996745765209198), (3, 0.9825855493545532), (4, 0.5611887574195862)]


# Rerank using llm

# Evaluation


In [15]:
# Evaluation
import json

path = "ALQAC_2025_data/alqac25_train.json"

with open(path, "r", encoding="utf-8") as f:
    json_data = json.load(f)

json_data[:2]

[{'question_id': 'train_alqac25_1',
  'question_type': 'Đúng/Sai',
  'text': 'Người nghiện ma túy từ đủ 18 tuổi trở lên bị áp dụng biện pháp xử lý hành chính đưa vào cơ sở cai nghiện bắt buộc theo quy định của Luật Xử lý vi phạm hành chính khi bị phát hiện sử dụng chất ma túy một cách trái phép trong thời gian cai nghiện ma túy tự nguyện, đúng hay sai?',
  'relevant_articles': [{'law_id': 'Luật Phòng, chống ma túy',
    'article_id': '32'}],
  'answer': 'Đúng'},
 {'question_id': 'train_alqac25_2',
  'question_type': 'Đúng/Sai',
  'text': 'Quan hệ hôn nhân và gia đình có yếu tố nước ngoài là quan hệ hôn nhân và gia đình mà ít nhất một bên tham gia là người nước ngoài, người Việt Nam định cư ở nước ngoài, đúng hay sai?',
  'relevant_articles': [{'law_id': 'Luật Hôn nhân và gia đình',
    'article_id': '3'}],
  'answer': 'Đúng'}]

In [ ]:
def calculate_precision(retrieved_articles, relevant_articles):
    """
    Calculates precision for a single question.
    A retrieved article is correct if its (law_id, article_id) tuple matches a relevant article.
    """
    retrieved_set = {(item["document"], item["id"]) for item in retrieved_articles}
    relevant_set = {(item["law_id"], item["article_id"]) for item in relevant_articles}

    correctly_retrieved = len(retrieved_set.intersection(relevant_set))
    total_retrieved = len(retrieved_set)

    if total_retrieved == 0:
        return 0.0

    return correctly_retrieved / total_retrieved


def calculate_recall(retrieved_articles, relevant_articles):
    """
    Calculates recall for a single question.
    A retrieved article is correct if its (law_id, article_id) tuple matches a relevant article.
    """
    retrieved_set = {(item["document"], item["id"]) for item in retrieved_articles}
    relevant_set = {(item["law_id"], item["article_id"]) for item in relevant_articles}

    correctly_retrieved = len(retrieved_set.intersection(relevant_set))
    total_relevant = len(relevant_set)

    if total_relevant == 0:
        return 0.0

    return correctly_retrieved / total_relevant


def calculate_f2_score(precision, recall):
    """
    Calculates the F2 score based on the provided formula.
    """
    if (4 * precision + recall) == 0:
        return 0.0

    return (5 * precision * recall) / (4 * precision + recall)


In [ ]:
import os
import json

all_precision = []
all_recall = []
all_f2_scores = []
details = []

os.makedirs("evaluation", exist_ok=True)

for idx, item in enumerate(json_data):
    query = item["text"]
    relevant_articles = item["relevant_articles"]

    retrieved_results = retrieve(query)
    retrieved_articles = retrieved_results.get("result", [])
    precision = calculate_precision(retrieved_articles, relevant_articles)
    recall = calculate_recall(retrieved_articles, relevant_articles)
    f2 = calculate_f2_score(precision, recall)

    all_precision.append(precision)
    all_recall.append(recall)
    all_f2_scores.append(f2)
    details.append(
        {
            "question_id": item.get("question_id", idx),
            "query": query,
            "precision": precision,
            "recall": recall,
            "f2_score": f2,
            "retrieved_articles": retrieved_articles,
            "relevant_articles": relevant_articles,
        }
    )

if all_f2_scores:
    average_f2 = sum(all_f2_scores) / len(all_f2_scores)
    average_precision = sum(all_precision) / len(all_precision)
    average_recall = sum(all_recall) / len(all_recall)
    print(f"Average Precision: {average_precision:.4f}")
    print(f"Average Recall: {average_recall:.4f}")
    print(f"Average F2-Score: {average_f2:.4f}")
else:
    print("Could not calculate F2-Score, no data processed.")

with open(
    f"evaluation/detailed_metrics_{TOP_K}_{SPARSE_TOP_K}.json", "w", encoding="utf-8"
) as f:
    json.dump(
        {
            "average_precision": average_precision if all_f2_scores else None,
            "average_recall": average_recall if all_f2_scores else None,
            "average_f2_score": average_f2 if all_f2_scores else None,
            "details": details,
        },
        f,
        ensure_ascii=False,
        indent=2,
    )
